## Compare SOC dependency on SEI model

In [95]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl; 
fs=17
font = {'family' : 'DejaVu Sans',  'size'   : fs}
mpl.rc('font', **font)

In [96]:
# task: post-processing - adjust to make the cell degrade to a reasonable value
variables = [
    "Time [h]",
    "Discharge capacity [A.h]",
    "Positive electrode capacity [A.h]",
    "Negative electrode capacity [A.h]",
    "Loss of capacity to negative SEI [A.h]",
    "Total lithium capacity in particles [A.h]",

    "Positive electrode stoichiometry",
    "Negative electrode stoichiometry",
]
Short_ylabel ={
    "Discharge capacity [A.h]": "Cap / A.h" ,
    "Positive electrode capacity [A.h]": "Cap_Pe / A.h",
    "Negative electrode capacity [A.h]": "Cap_Ne / A.h",
    "Loss of capacity to negative SEI [A.h]": "LLI to SEI / A.h",
    "Total lithium capacity in particles [A.h]": "Li in Particle / A.h" ,
    "Positive electrode stoichiometry Start": "PE Sto Start" ,
    "Negative electrode stoichiometry Start": "NE Sto Start" ,
    "Positive electrode stoichiometry End": "PE Sto End" ,
    "Negative electrode stoichiometry End": "NE Sto End" ,
}

def Run_Age_w_RPT(model, param, variables, age_T, total_cycles, num_one_age_set, calendar_soc):
    # age_T is the ageing temperature in degC
    # calendar_soc must be between 0~100%
    
    dis_time_in_h = (100 - calendar_soc)/100 * 5 / (0.3*5) 
    if calendar_soc == 100:
        str_step = "Rest for 1 s"
    else:
        str_step = f"Discharge at 0.3C for {dis_time_in_h} hour or until 2.5 V"
    num_loop = total_cycles / num_one_age_set
    age_experiment = pybamm.Experiment(
        [
            (        
                "Rest for 432 hours (24 hours period)", # 342 for 9.5 months
                # "Rest for 1 hours",
            )
        ] * num_one_age_set,   
    )
    rpt_experiment = pybamm.Experiment(
        [
            (        
                "Charge at 0.3C until 4.2 V",
                "Hold at 4.2 V until C/50",
                "Rest for 30 minutes",
                "Discharge at 0.3C until 2.5 V",
                "Rest for 30 minutes",
                "Charge at 0.3C until 4.2 V",
                "Hold at 4.2 V until C/50",
                str_step,
            )
        ] * 1,   
    )
    param["Initial temperature [K]"] = 273.15 + 25
    param["Ambient temperature [K]"] = 273.15 + 25
    def Get_customized_dict_from_sol_step(Sol_dict, step, variables, cyc_No):  
        Sol_dict["cyc no."].append(cyc_No)
        for var in variables:
            if var in ["Discharge capacity [A.h]"]:
                Q_dis_cyc = step["Discharge capacity [A.h]"].entries
                Q_dis = abs(Q_dis_cyc[-1] - Q_dis_cyc[0])
                Sol_dict[var].append(Q_dis)
            elif var in ["Positive electrode stoichiometry","Negative electrode stoichiometry"]:
                Sol_dict[var + " Start"].append(
                    step[var].entries[0])
                Sol_dict[var + " End"].append(
                    step[var].entries[-1])
            else:
                Sol_dict[var].append(step[var].entries[0] )
        return Sol_dict
    # run the process break in cycle
    sim = pybamm.Simulation(
        model, experiment=rpt_experiment, 
        parameter_values=param,
    )
    sol_rpt = sim.solve()
    
    Sol_dict = {}
    Sol_dict["cyc no."] = []
    for var in variables:
        if var in ["Positive electrode stoichiometry","Negative electrode stoichiometry"]:
            Sol_dict[var + " Start"] = []
            Sol_dict[var + " End"] = []
        else:
            Sol_dict[var] = []
    # post-process for break in / 1st RPT:
    Sol_dict = Get_customized_dict_from_sol_step(
        Sol_dict, sol_rpt.cycles[0].steps[3], variables, cyc_No=0)
    # now repeat:
    for i in range(int(num_loop)):
        # run age:
        param["Initial temperature [K]"] = 273.15 + age_T
        param["Ambient temperature [K]"] = 273.15 + age_T
        sim = pybamm.Simulation(
            model, experiment=age_experiment, 
            parameter_values=param,)
        sol_age = sim.solve(
            starting_solution=sol_rpt, save_at_cycles = num_one_age_set)
        # run rpt:
        param["Initial temperature [K]"] = 273.15 + 25
        param["Ambient temperature [K]"] = 273.15 + 25
        sim = pybamm.Simulation(
            model, experiment=rpt_experiment, 
            parameter_values=param,)
        sol_rpt = sim.solve(starting_solution=sol_age)
        Sol_dict = Get_customized_dict_from_sol_step(
            Sol_dict, sol_rpt.cycles[-1].steps[3], variables, 
            cyc_No=num_one_age_set*(i+1) )
    return Sol_dict


In [97]:
432*20/24/30

12.0

In [98]:
model = pybamm.lithium_ion.SPM({"SEI": "solvent-diffusion limited",})
for var, equation in model.initial_conditions.items():
    print(var)

Discharge capacity [A.h]
Throughput capacity [A.h]
X-averaged negative particle concentration [mol.m-3]
X-averaged positive particle concentration [mol.m-3]
X-averaged negative inner SEI thickness [m]
X-averaged negative outer SEI thickness [m]


In [99]:
param = pybamm.ParameterValues("Chen2020") 
options = [
    "solvent-diffusion limited", 
    "reaction limited",
    "electron-migration limited",
    "interstitial-diffusion limited",
]
keys = [
    "Outer SEI solvent diffusivity [m2.s-1]",
    "SEI reaction exchange current density [A.m-2]",
    "Inner SEI electron conductivity [S.m-1]",
    "Inner SEI lithium interstitial diffusivity [m2.s-1]",
]
values = [
    3e-20,
    1.15e-7,  # "reaction limited", 9.5e-8 is too small, 1.5e-7 is too big
    1e-9,
    2e-16
]

param["SEI growth activation energy [J.mol-1]"] =  5e4
total_cycles, num_one_age_set = 20, 1
age_T = 25
Calendar_soc = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90,  100]
Sol_Dict_List = {}
for option in options:
    Sol_Dict_List[option] =  []

for i in range(4):
    print()
    print(f"Running option {options[i]}")
    print()
    model = pybamm.lithium_ion.SPM({"SEI": options[i]})
    param[keys[i]] = values[i]
    for calendar_soc in Calendar_soc:
        print(f"Running calendar SOC ={calendar_soc}%")
        start_1 = pybamm.Timer() 
        Sol_Dict_List[options[i]].append(Run_Age_w_RPT(
            model, param, variables, age_T=age_T, 
            total_cycles=total_cycles, 
            num_one_age_set=num_one_age_set,
            calendar_soc=calendar_soc))
        print(f"running time: for option {options[i]}, "
              f"{calendar_soc}% SOC is {start_1.time()}")



Running option solvent-diffusion limited

Running calendar SOC =5%


running time: for option solvent-diffusion limited, 5% SOC is 40.446 s
Running calendar SOC =10%
running time: for option solvent-diffusion limited, 10% SOC is 36.362 s
Running calendar SOC =20%
running time: for option solvent-diffusion limited, 20% SOC is 37.483 s
Running calendar SOC =30%
running time: for option solvent-diffusion limited, 30% SOC is 35.866 s
Running calendar SOC =40%
running time: for option solvent-diffusion limited, 40% SOC is 35.090 s
Running calendar SOC =50%
running time: for option solvent-diffusion limited, 50% SOC is 35.996 s
Running calendar SOC =60%
running time: for option solvent-diffusion limited, 60% SOC is 35.101 s
Running calendar SOC =70%
running time: for option solvent-diffusion limited, 70% SOC is 35.914 s
Running calendar SOC =80%
running time: for option solvent-diffusion limited, 80% SOC is 34.427 s
Running calendar SOC =90%
running time: for option solvent-diffusion limited, 90% SOC is 35.613 s
Running calendar SOC =100%
running time: for op

In [100]:
# add soh_dict
soh_dict = {}
for option in options:
    soh_dict[option] =  []
for option in options:
    for Sol_dict in Sol_Dict_List[option]:
        soh_dict[option].append(
            (
                Sol_dict["Discharge capacity [A.h]"][-1] 
                / Sol_dict["Discharge capacity [A.h]"][0]) * 100
        ) 

In [102]:
import os
str_path_0 = os.path.abspath(os.path.join(pybamm.__path__[0],'../..'))
Save_fig_path = str_path_0 + "/examples/Reproduce_Paper"

In [103]:
Save_fig_path

'/home/ruihe_li/EnvPB_Linux/PyBaMM/examples/Reproduce_Paper'

In [104]:
# plot SOC dependency:
labels = [
    "solvent-diffusion", 
    "reaction",
    "electron-migration",
    "interstitial-diffusion",
]
fig, axs = plt.subplots( figsize=(10,5),tight_layout=True)
for option, label in zip(options, labels):
    axs.plot(
        Calendar_soc, soh_dict[option], "-o", 
        label = label)
axs.set_ylabel("SOH after 1 year / %")
axs.set_xlabel("Calendar SOC / %")
axs.legend(prop={'size':fs-3}, loc='best')
axs.legend(loc='center left', bbox_to_anchor=(1, 0.5))
fig.savefig(Save_fig_path + "/Sweep SEI submodel store at different SOC.svg")